<a href="https://colab.research.google.com/github/roshanappa/Deep-Learning-Models./blob/master/First_Artificial_Neural_Network_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#data preprocessing.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard

In [0]:
import pickle
import time

In [0]:
NAME = 'Simple ANN Model {}'.format(int(time.time()))

In [0]:
tensorboard = TensorBoard(log_dir='logs/ {}'.format(NAME))

## Data Preprocessing.

In [0]:
data = pd.read_csv('/content/Churn_Modelling.csv')

In [0]:
pd.pandas.set_option('display.max_columns',None)

In [0]:
data.head()

In [0]:
data.drop(columns=['RowNumber','CustomerId'],inplace=True)

In [0]:
data.tail()

In [0]:
data.columns

In [0]:
data.isnull().sum()

In [0]:
dataframe = pd.DataFrame({'Dtypes':data.dtypes,'Nuique':data.nunique(),'Null_values':data.isnull().sum(),'Duplicated':data.duplicated().sum()})

In [0]:
dataframe

In [0]:
data.drop(columns=['Surname'],inplace=True)

In [0]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [0]:
data['Geography'].value_counts()

In [0]:
data['Geography'].value_counts().plot(kind='bar')
plt.show()

In [0]:
d = {'France':0,'Germany':1,'Spain':3}
data['Geography']=data['Geography'].map(d)

In [0]:
data['Gender'].value_counts().plot(kind='bar')
plt.show()

In [0]:
z = {'Male':0,'Female':1}
data['Gender']=data['Gender'].map(z)

## Prepare for the model.

In [0]:
data.head()

Here,we are getting the all the independed variables that can makes the impact on the dependent variable.But they are avalible in the in diffrent scale.So we to convert the in same scale by the Standarscaler.

In [0]:
feature = data.columns
feature

In [0]:
x = data.drop(columns=['Exited'])
y = data['Exited']

In [0]:
x.shape,y.shape

In [0]:

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.80,random_state=42,stratify=y)

In [0]:
x_train.shape,x_test.shape

In [0]:
y_train.value_counts()

In [0]:
scaler = StandardScaler()
X_train  = scaler.fit_transform(x_train)
X_test   = scaler.transform(x_test)

In [0]:
from imblearn.over_sampling import SMOTE

In [0]:
sm = SMOTE(sampling_strategy=1,random_state=42,k_neighbors=5)

In [0]:
x_train_res,y_train_res = sm.fit_sample(X_train,y_train)

In [0]:
print('0 with y_train_res :-',len(y_train_res[y_train_res==0]))
print('1 with y_train_res :-',len(y_train_res[y_train_res==1]))

In [0]:
x_train_res.shape

## Build An ANN Model.

In [0]:
model = Sequential()

In [0]:
model.add(Dense(120,activation='relu',kernel_initializer='uniform',input_shape=[x_train_res.shape[1]]))

In [0]:
model.add(Dense(60,activation='relu',kernel_initializer='uniform'))

In [0]:
model.add(Dense(1,activation='sigmoid',kernel_initializer='uniform'))

## Model Compile.

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
history=model.fit(x_train_res,y_train_res,batch_size=10,epochs=20,verbose=1,validation_split=0.20)

- In the above we calculated the model accuarcy that is 84% and validation accuarcy is 75%

In [0]:
val_loss,val_acc=model.evaluate(X_test,y_test)

- In the above the model is calculated the accuarcy on the basis of test data,From the above we can say that the diffrence between the training data and testing is not so much high.
- In this we can see the accuarcy on training data is good but the testing is equivalent to training data but acceptable because the diffrence is less.

In [0]:
y_pred = model.predict(X_test).ravel()
y_pred = pd.Series(y_pred)
#In this case we will get the probablities of the cust will exit or not.

In [0]:
index= np.where(y_pred>=0.5)

In [0]:
index

In [0]:
#If anyone cust have the probalities is greater than the 0.5 that's there is 50% chances to exit the bank.

In [0]:
Y_prediction=y_pred[y_pred>0.5]

- From the below we can understand,According to ranking avaliable in side data,are only those customer who more than 50% chances to leave the bank.

In [0]:
Y_prediction

In [0]:
predict_clases = model.predict_classes(X_test)
predict_clases

In [0]:
## From the below we can undestand who will stay and leave with the specified class.

- From the below we can easily understand we will stay and who will leave.

In [0]:
predicted_classes = pd.Series(predict_clases.ravel())
predicted_classes

## Confusion Matrix.

In [0]:
from sklearn.metrics import confusion_matrix,classification_report

In [0]:
mat = confusion_matrix(y_test,predicted_classes)
mat

In [0]:
sns.heatmap(mat,annot=True,fmt='d',linewidths=0.4,linecolor='black',xticklabels=['Stay','EXit'],yticklabels=['Stay','Exit'])
plt.show()

## Accuracy of the Model.

In [0]:
from sklearn.metrics import accuracy_score

- Accuary is nothing the Total number of the actual predicted classes.
- To the Adding of all.

In [0]:
accuracy = round((1398+246)/(1398+246+161+195)*100,1)  #(TP+TN)?(TP+TN+FP+FN)
print('Accuracy in % :-',accuracy)

In [0]:
print('Accuarcy Score :-',round(accuracy_score(y_test,predicted_classes),1)*100)

## Let's Calculate Result of Prediction on the Following data.

- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: $60000

- Number of Products: 2
- Does this customer have a credit card ? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: $50000

In [0]:
final_prediction=model.predict_classes([[0,600,0,40,3,60000,2,1,1,50000]])

### From the above we say,understand the according the above paramerter.There is fair chances of customer will not **exit** the bank.

## Cross Validate the Model.
- Because every stage of run the model will gives us the diffrent accuracy.
- We do not know which is accaracy is model accutually giving.
- So,Therefore we need the cross validation to check the model performance of the model at the diffrent cv.

In [0]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
def build_classifier():  #global variable
  model = Sequential()
  model.add(Dense(120,activation='relu',kernel_initializer='uniform',input_shape=[x_train_res.shape[1]]))
  model.add(Dense(60,activation='relu',kernel_initializer='uniform'))
  model.add(Dense(1,activation='sigmoid',kernel_initializer='uniform'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model   #local variable.
model

model = KerasClassifier(build_classifier,batch_size=10,epochs=20)
accuaracies = cross_val_score(model,x_train_res,y_train_res,cv=10,n_jobs=-1)

### Under the Cross Validation the accuarcy is calculated for the CV=10,but if we see the diffrence between is so much high means our accuarcy pretty much genuine.


In [0]:
print(accuaracies)

In [0]:
print('Mean accuarcy of the Calculated Accuaracy :-',accuaracies.mean())
print('Standard Deviation(Variance)  of the Calculated Accuaracy :-',accuaracies.std())

mean and Standard variance is not so much affeted by bias...
because we having the very low std deviation.

## Relationship with the Validation Score,loss and Accuarcy Score,loss

In [0]:
history.history

In [0]:
plt.plot(range(1,21),history.history['accuracy'])
plt.plot(range(1,21),history.history['val_accuracy'])
plt.xlabel('Epoches')
plt.ylabel('Accuarcy')
plt.title('Relational Graph between the Model and validation accuarcy')
plt.legend(['Model Accuarcy','Validation Accuarcy'],loc='best')
plt.show()
print()
plt.plot(range(1,21),history.history['loss'])
plt.plot(range(1,21),history.history['val_loss'])
plt.xlabel('Epoches')
plt.ylabel('losses')
plt.title('Relational Graph between the Model and validation losse')
plt.legend(['Model loss','Validation loss'],loc='best')
plt.show()

## Thank You !!